In [165]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [166]:
import re

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [167]:
class Computer():
    def __init__(self,a,b,c,instructions):
        # Registers
        self.a = a
        self.b = b
        self.c = c

        # Instructions
        self.instructions = [int(x) for x in instructions if x != ',']
        
        # Misc
        self.num_instructions = len(self.instructions)
        self.instr_pointer = 0
        self.instr = int(instructions[0])
        self.operand = int(instructions[2])
        self.outputs = []
    
    def print_regs(self):
        '''
        Prints current register values for debugging purposes
        '''
        print(f'A={self.a}\nB={self.b}\nC={self.c}\n')
    
    def combo_operand(self, operand):
        '''
        Returns combo value of given operand
        '''
        match operand:
            case 0 | 1 | 2 | 3:
                combo = operand
            case 4:
                combo = self.a
            case 5:
                combo = self.b
            case 6:
                combo = self.c
            case 7:
                raise(ValueError('Operand cannot be 7 in adv'))
            
        return combo
    
    def adv(self, operand):
        '''
        Register A = register A // 2^(combo operand)
        '''
        denom_power = self.combo_operand(operand)
        
        self.a = self.a // (2**denom_power)
    
    def bxl(self, operand):
        '''
        Register B = Bitwise XOR of register B and literal operand
        '''
        self.b = self.b ^ operand
    
    def bst(self, operand):
        '''
        Register B = combo operand mod 8
        '''
        self.b = self.combo_operand(operand) % 8
    
    def jnz(self, operand):
        '''
        Does nothing if register A = 0, else 'jumps' by setting instruction pointer to literal operand
        '''
        if self.a:
            self.instr_pointer = operand
    
    def bxc(self, operand):
        '''
        Register B = Bitwise XOR of Registers B and C
        '''
        self.b = self.b ^ self.c
    
    def out(self, operand):
        '''
        Outputs value(s) of combo operand modulo 8
        '''
        self.outputs.append((''.join([x+',' for x in (str(self.combo_operand(operand) % 8))]))[:-1])

    def bdv(self, operand):
        '''
        Register B = register A // 2^(combo operand)
        '''
        denom_power = self.combo_operand(operand)
        
        self.b = self.a // (2**denom_power)

    def cdv(self, operand):
        '''
        Register C = register A // 2^(combo operand)
        '''
        denom_power = self.combo_operand(operand)
        
        self.c = self.a // (2**denom_power)
    
    def read_program(self):
        '''
        Applies the instructions given by the program
        '''
        functions = [self.adv, self.bxl, self.bst, self.jnz, self.bxc, self.out, self.bdv, self.cdv]

        while (self.instr_pointer < self.num_instructions-1):
            self.instr = self.instructions[self.instr_pointer]
            self.operand = self.instructions[self.instr_pointer+1]

            fn = functions[self.instr]

            fn(self.operand)

            #self.print_regs()

            if self.instr != 3 or (self.instr == 3 and self.a == 0):
                self.instr_pointer += 2
            
    

        

In [168]:
reg_info, program_info = input.split('\n\n')
a_info, b_info, c_info = reg_info.split('\n')

a_init = int(re.findall(r': (\d*)', a_info)[0])
b_init = int(re.findall(r': (\d*)', b_info)[0])
c_init = int(re.findall(r': (\d*)', c_info)[0])
instructions_init = re.findall(r': ([\d,]*)', program_info)[0]

computer = Computer(a_init,b_init,c_init,instructions_init)
computer.read_program()
','.join(computer.outputs)


'7,1,3,7,5,1,0,3,4'

# Part 2

In [169]:
reg_info, program_info = input.split('\n\n')
a_info, b_info, c_info = reg_info.split('\n')

a_init = int(re.findall(r': (\d*)', a_info)[0])
b_init = int(re.findall(r': (\d*)', b_info)[0])
c_init = int(re.findall(r': (\d*)', c_info)[0])
instructions_init = re.findall(r': ([\d,]*)', program_info)[0]

Analysing the puzzle input's instructions, the outputs come from the following procedure:

- Take the last 3 bits of A
- Swap the middle bit
- Swap the bits with XYZ, where X,Y,Z are the (B-3 -> Bth) last respectively bits of A
- Swap every bit

Then remove last 3 bits of A and repeat procedure again

We can work out the binary digits if we work from the last output backwards (since for the last output we know XYZ = 000). Still need to iterate through every value but this reduces the seach space to just the numbers 1-8 for each output.

In [170]:
reversed_outputs = [int(x) for x in re.findall(r'\d',instructions_init)]
reversed_outputs.reverse()

bin_str = '00000000'

for desired_output in reversed_outputs:
    for num in range(8):
        bin_num_str= format(num, '03b')
        cur_bin_str = bin_str+bin_num_str
        cur_a = int(cur_bin_str,2)

        computer = Computer(cur_a,b_init,c_init,instructions_init)
        computer.read_program()
        #print(computer.outputs)
        if computer.outputs[0] == str(desired_output):
            bin_str += bin_num_str
            break
    #print('\n')
        
int(bin_str,2)

190384113204239

In [ ]:
# Check:

cur_a = 190384113204239
computer = Computer(cur_a,b_init,c_init,instructions_init)
computer.read_program()
#print(','.join(computer.outputs))